In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,sys
import argparse
import cv2
import pickle
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import precision_recall_fscore_support

In [2]:
curr_dir = os.getcwd()
rel_path = os.path.join(curr_dir, "output")
rel_path

'D:\\Nikita\\output'

In [3]:
def raw_shapes_to_edges(path):
    data = []
    for elem in os.listdir(path):
        path_orig = os.path.join(path,elem)
        
        img = cv2.imread(path_orig, cv2.IMREAD_COLOR)
        edges = cv2.Canny(img, 100, 200)
        kernel = np.ones((2, 2), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=2)
        data.append((elem.split("_")[0].lower(),edges))
    return data

In [12]:
data = raw_shapes_to_edges(rel_path)
with open("data.pickle", "wb") as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

KeyboardInterrupt: 

In [4]:
class ShapeDataset:

    def __init__(self, transform):
        with open("data.pickle", "rb") as handle:
            self.examples = pickle.load(handle)
            
        self.transform = transform
        self.classes = {'pentagon': 0,
                        'circle': 1,
                        'nonagon': 2,
                        'triangle': 3,
                        'octagon': 4,
                        'square': 5,
                        'heptagon': 6,
                        'hexagon': 7,
                        'star': 8}

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = Image.fromarray(self.examples[idx][1], 'L')
        tensor_image = self.transform(image)
        torch.set_printoptions(profile="full")
        sample = {"label": self.classes[self.examples[idx][0]], "picture": tensor_image}
        return sample

    def train_test_dataset(self, test_split=0.2):
        train_size = 1-test_split
        subsets = torch.utils.data.random_split(self, [int(len(self)*train_size),int(len(self)*test_split)])
        datasets = {'train': subsets[0],
                    'test': subsets[1]}
        return datasets


In [69]:
class TestValidate:

    def __init__(self, device, batch_size, criterion, net):
        self.device = device
        self.batch_size = batch_size
        self.criterion = criterion
        self.net = net

    def eval(self, dataloader):
        def score_to_modality(scores: torch.Tensor):
            tensor_list = scores.tolist()
            modality = []
            for row in tensor_list:
                modality.append(row.index(max(row)))
            return modality

        with torch.no_grad():
            t_correct = 0
            t_total = 0
            total_per_mode = [0] * 9
            correct_per_mode = [0] * 9
            val_losses = []

            all_labels = []
            all_predicted = []

            for i, data in enumerate(dataloader):
                inputs, labels = data["picture"], data["label"]
                inputs = inputs.to(self.device)

                outputs = self.net(inputs)
                labels = labels.view(self.batch_size, -1).squeeze(1).long().to(self.device)
                loss = self.criterion(outputs.view(self.batch_size, -1), labels)
                val_losses.append(loss.item())
                predicted = score_to_modality(outputs.view(self.batch_size, -1))

                all_labels.extend(labels)
                all_predicted.extend(predicted)

                for o, elem in enumerate(predicted):
                    total_per_mode[int(labels[o])] += 1
                    if labels[o] == predicted[o]:
                        correct_per_mode[predicted[o]] += 1
                        t_correct += 1
                    t_total += 1

            mode_statistics = []
            for k in range(len(correct_per_mode)):
                if correct_per_mode[k] == 0 or total_per_mode[k] == 0:
                    mode_statistics.append(0)
                    continue
                mode_statistics.append(1 / (total_per_mode[k] / correct_per_mode[k]))
            all_labels = [int(x) for x in all_labels]
            precision, recall, fbeta, _ = precision_recall_fscore_support(all_labels,all_predicted,average='macro')

            print('Accuracy: %d %%' % (100 * t_correct / t_total))
            print("Loss: {:.6f}".format(np.mean(val_losses)))
            print("Mode-correct:")
            print(total_per_mode)
            print(mode_statistics)
            print("Precision: {:.3f}, Recall: {:.3f}, F-beta: {:.3f}".format(precision,recall,fbeta))

In [6]:
from torch import nn
import torch.nn.functional as F

class ShapesCNN(nn.Module):
    def __init__(self):
        super(ShapesCNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=2)
        self.conv31 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=2)
        self.pool3 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.conv4 = nn.Conv2d(128, 196, kernel_size=3, stride=1, padding=2)
        self.pool4 = nn.MaxPool2d(kernel_size=4, stride=2)

        self.fc1 = nn.Linear(7056, 2048)
        self.fc2 = nn.Linear(2048, 9)

        self.drop = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv31(x))
        x = self.pool3(x)

        x = F.relu(self.conv4(x))
        x = self.pool4(x)

        x = x.reshape(x.size(0), -1)
        x = self.drop(x)
        x = F.relu(self.fc1(x))

        x = self.fc2(x)

        return x


# class ShapesCNN(nn.Module):
#     def __init__(self):
#         super(ShapesCNN, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, 1)
#         self.conv2 = nn.Conv2d(32, 64, 3, 1)
#         self.dropout1 = nn.Dropout(0.25)
#         self.dropout2 = nn.Dropout(0.5)
#         self.fc1 = nn.Linear(64, 128)
#         self.fc2 = nn.Linear(128, 9)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = F.max_pool2d(x, 2)
#         x = self.dropout1(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.dropout2(x)
#         x = self.fc2(x)
#         return x

In [7]:
batch_size = 256
losses = []
running_loss = 0
val_counter = 0
epochs = 10

transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize(100),
                                transforms.ToTensor(),
                                transforms.Normalize((0,), (1,))])
ds = ShapeDataset(transform)
datasets = ds.train_test_dataset(0.2)
dataloaders = {x: DataLoader(datasets[x], batch_size, shuffle=True, num_workers=0, drop_last=True) for x in
               ['train', 'test']}

print("{}, {}, Train, Test Samples".format(len(datasets['train']),
                                                      len(datasets['test'])))

net = ShapesCNN()
net.cuda()
net.train()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
testval = TestValidate(device,batch_size,criterion,net)

for epoch in range(epochs):
    for i, data in enumerate(dataloaders["train"], 0):
        val_counter += 1

        inputs, labels = data["picture"], data["label"]
        inputs = inputs.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        outputs = outputs.to(device)

        labels = labels.view(batch_size, -1).squeeze(1).long().to(device)
        loss = criterion(outputs.view(batch_size, -1), labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 0:
            losses.append(running_loss / 10)
            print('[%d, %5d] loss: %.3f' % (epoch, i, losses[-1]))
            running_loss = 0.0

72000, 18000, Train, Test Samples
[0,     0] loss: 0.220
[0,    10] loss: 2.210
[0,    20] loss: 2.187
[0,    30] loss: 2.091
[0,    40] loss: 1.952
[0,    50] loss: 1.927
[0,    60] loss: 1.907
[0,    70] loss: 1.774
[0,    80] loss: 1.648
[0,    90] loss: 1.819
[0,   100] loss: 1.694
[0,   110] loss: 1.484
[0,   120] loss: 1.398
[0,   130] loss: 1.248
[0,   140] loss: 1.160
[0,   150] loss: 1.196
[0,   160] loss: 1.073
[0,   170] loss: 0.877
[0,   180] loss: 0.831
[0,   190] loss: 0.725
[0,   200] loss: 0.808
[0,   210] loss: 0.675
[0,   220] loss: 0.583
[0,   230] loss: 0.536
[0,   240] loss: 0.623
[0,   250] loss: 0.642
[0,   260] loss: 0.511
[0,   270] loss: 0.442
[0,   280] loss: 0.529
[1,     0] loss: 0.059
[1,    10] loss: 0.454
[1,    20] loss: 0.389
[1,    30] loss: 0.335
[1,    40] loss: 0.333
[1,    50] loss: 0.305
[1,    60] loss: 0.324
[1,    70] loss: 0.324
[1,    80] loss: 0.307
[1,    90] loss: 0.280
[1,   100] loss: 0.336
[1,   110] loss: 0.355
[1,   120] loss: 0.311


In [70]:
testval.eval(dataloaders["test"])

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [8]:
torch.save(net.state_dict(), "shapes_classifier_cnn2.pt")

In [8]:
print(transform)

Compose(
    Grayscale(num_output_channels=1)
    Resize(size=100, interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=(0,), std=(1,))
)


In [24]:
def predict(image_path, model, topk=9):
    model.eval()
    model.cpu()
    img = transform(Image.open(image_path))
    img = img.unsqueeze_(1)
    #img = img.float()
    
    with torch.no_grad():
        output = model.forward(img)
        probs, classes = torch.topk(input=output, k=topk)
        top_prob = probs.exp()
    return probs, classes

tensor([[8, 4, 2, 6, 1, 3, 5, 7, 0]])

In [52]:
with torch.no_grad():
    index = 2333
    item = datasets['test'][index]
    image = item['picture'].unsqueeze_(0)
    true_target = item['label']
    prediction = net(image)
    predicted_class = np.argmax(prediction)
    print("prediction: ",predicted_class," Actual :", true_target)

prediction:  tensor(5)  Actual : 5


In [1]:
plt.plot(losses)
plt.xlabel('epoch')
plt.ylabel('loss')

NameError: name 'plt' is not defined